In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.impute import KNNImputer
import missingno as msno
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.linear_model import LinearRegression

## Data Understanding

In [ ]:
hitters = pd.read_csv("../input/regularization-of-hitters/Hitters.csv")

In [ ]:
df = hitters.copy()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df[df["Hits"] < 70]

In [ ]:
df[df["Hits"] > 70]

In [ ]:
df[df["Salary"].isnull()].head(10)

In [ ]:
sns.lineplot(x = "Salary",y = "Years",data= df,hue = "League",style = "Division")

In [ ]:
df.describe([0.01,0.25,0.75,0.99]).T

In [ ]:
f, ax = plt.subplots(figsize= [20,15])
sns.heatmap(df.corr(), annot=True, fmt=".2f", ax=ax, cmap = "coolwarm" )
ax.set_title("Correlation Matrix", fontsize=20)
plt.show()

In [ ]:
df.groupby(["League"])["Salary"].mean()

In [ ]:
df["Salary"].mean()

# Data Preparation

In [ ]:
msno.matrix(df);

In [ ]:
for i in ["Hits","HmRun","Runs","RBI","Walks","Years","CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks","PutOuts","Assists","Errors","Salary","AtBat"]:

    Q1 = df[i].quantile(0.25)
    Q3 = df[i].quantile(0.75)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[i] > upper) | (df[i] < lower)].any(axis=None):
        print(i,"yes")
        print(df[(df[i] > upper) | (df[i] < lower)].shape[0])
    else:
        print(i, "no")


# One hot encoding with pd lib

In [ ]:
df = pd.get_dummies(df, columns =["League","Division","NewLeague"], drop_first = True)

In [ ]:
cols = df.columns

# Missing values

In [ ]:
imputer = KNNImputer(n_neighbors=6)
df_filled = imputer.fit_transform(df)

# Outliers 

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
clf =LocalOutlierFactor(n_neighbors= 20,contamination= 0.1)

In [ ]:
clf.fit_predict(df_filled)

In [ ]:
df_scores = clf.negative_outlier_factor_

In [ ]:
np.sort(df_scores)[0:30]

In [ ]:
th = np.sort(df_scores)[8]
th

In [ ]:
outlier = df_scores > th

In [ ]:
dff = df_filled[df_scores > th]

In [ ]:
dff = pd.DataFrame(dff,columns = cols)

# PREDICTION

# Linear Regression

In [ ]:

from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
scaler = StandardScaler()
y = dff["Salary"]
X = dff.drop('Salary', axis=1)


In [ ]:
df_ = pd.DataFrame(dff, columns = cols)

In [ ]:
y.head()

In [ ]:
X.head()

In [ ]:
dummies = dff[["League_N","Division_W","NewLeague_N"]]
dummies

In [ ]:
X = X.drop(["League_N","Division_W","NewLeague_N"],axis = 1)

In [ ]:
X.head()

In [ ]:
cols = X.columns

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
X = pd.DataFrame(X, columns = cols)

In [ ]:
X.head()

In [ ]:
X_ = pd.concat([X,dummies],axis = 1)

In [ ]:
X_

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_, 
                                                    y, 
                                                    test_size = 0.20, random_state = 46)

In [ ]:
X_.head()

In [ ]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

In [ ]:
#train
y_pred = reg_model.predict(X_train)
np.sqrt(mean_squared_error(y_train, y_pred))

In [ ]:
#test
y_pred = reg_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

### Tuned Linear R

In [ ]:
linear_tuned = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))

In [ ]:
y_pred = reg_model.predict(X_test)

In [ ]:
linear_sc = np.sqrt(mean_squared_error(y_test, y_pred))

## Ridge Regression

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.linear_model import Ridge, Lasso

In [ ]:
ridge_model = Ridge().fit(X_train,y_train)

In [ ]:
## train
y_pred = ridge_model.predict(X_train)
np.sqrt(mean_squared_error(y_train, y_pred))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV, LassoCV

In [ ]:
#test
y_pred = ridge_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))


In [ ]:
alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]

In [ ]:
ridge_cv = RidgeCV(alphas = alphas, scoring = "neg_mean_squared_error", cv = 10,normalize = True).fit(X_train,y_train)

In [ ]:
ridge_cv.alpha_

# Tuned Ridge

In [ ]:
ridge_tuned = Ridge(alpha =ridge_cv.alpha_).fit(X_train,y_train)

In [ ]:
#test
y_pred = ridge_tuned.predict(X_test)
ridge_sc = np.sqrt(mean_squared_error(y_test, y_pred))
ridge_sc

# Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]

In [ ]:
lasso_model = Lasso().fit(X_train, y_train)

In [ ]:
#train
y_pred  = lasso_model.predict(X_train)
np.sqrt(mean_squared_error(y_train,y_pred))

In [ ]:
#test
y_pred  = lasso_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
lasso_cv_model = LassoCV(alphas = alphas, cv = 10,max_iter = 10000).fit(X_train, y_train)

In [ ]:
lasso_cv_model.alpha_

# Tuned Lasso

In [ ]:
lasso_tuned = Lasso(alpha =lasso_cv_model.alpha_).fit(X_train,y_train)

In [ ]:
y_pred = lasso_tuned.predict(X_test)

In [ ]:
lasso_sc =np.sqrt(mean_squared_error(y_test,y_pred))
lasso_sc

# ElasticNET

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV,ElasticNetCV
from sklearn.linear_model import ElasticNet

In [ ]:
enet_model = ElasticNet().fit(X_train, y_train)
y_pred = enet_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
enet_params = {"l1_ratio": [0.001,0.01,0.1,0.4,0.5,0.6,0.8,1],
              "alpha":[0.1,0.01,0.001,0.2,0.3,0.5,0.8,0.9,1],
              "max_iter" :[1000,5000,10000]}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gs_cv_enet = GridSearchCV(enet_model, enet_params, cv = 10,n_jobs = -1).fit(X_train, y_train)

In [ ]:
gs_cv_enet.best_params_

# Tuned ElasticNET

In [ ]:
enet_tuned = ElasticNet(**gs_cv_enet.best_params_,normalize = True)

In [ ]:
enet_tuned = ElasticNet().fit(X_train, y_train)
y_pred = enet_tuned.predict(X_test)
enet_sc = np.sqrt(mean_squared_error(y_test, y_pred))
enet_sc

# Reporting

In [ ]:
models = pd.DataFrame({"Model" : ["Linear","Rigde","Lasso","ElasticNET"],
                     "Score" : [linear_sc,ridge_sc,lasso_sc,enet_sc]})

In [ ]:
models

In [ ]:
plt.plot(models["Model"],models["Score"], 'ro');
plt.title("Regularization")
plt.xlabel("Models")
plt.ylabel("Scores")
plt.show()